### Caution

Adam + Half Precision causes zero devision somewhere(eps might be the culprit...)

* [Adam+Half Precision = NaNs?](https://discuss.pytorch.org/t/adam-half-precision-nans/1765/7)

In [0]:
!pip install geffnet
!pip uninstall tensorboard
!pip install --force-reinstall tf-nightly-2.0-preview

In [0]:
!pip install --upgrade efficientnet-pytorch

Requirement already up-to-date: efficientnet-pytorch in /usr/local/lib/python3.6/dist-packages (0.6.1)


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip drive/My\ Drive/cifar.zip

In [0]:
import torch
import torchvision
import glob
import time
import PIL
import geffnet
import datetime
import os
from torch.utils.tensorboard import SummaryWriter
from efficientnet_pytorch import EfficientNet
from collections import OrderedDict

In [0]:
batch_size = 512
num_workers=4

train_imgs = glob.glob('cifar/train/*.png')
test_imgs = glob.glob('cifar/test/*.png')
with open('cifar/labels.txt','r') as f:
  l = f.readlines()
label_idx_dict = {label.replace('\n',''):idx for idx, label in enumerate(l)}

transforms_dict = {'train':
                   torchvision.transforms.Compose([
                                                   torchvision.transforms.RandomResizedCrop(32),
                                                   torchvision.transforms.RandomHorizontalFlip(),
                                                   torchvision.transforms.ToTensor(),
                                                   torchvision.transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
                   ])
                   ,'test':torchvision.transforms.Compose([
                                                           torchvision.transforms.ToTensor(),
                                                           torchvision.transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
                   ])
                   }

class CIFAR10Dataset(torch.nn.Module):
  def __init__(self,img_path, label_idx_dict, transforms=None):
    self.img_path = img_path
    self.label_idx_dict = label_idx_dict
    self.transforms = transforms
    self.length = len(self.img_path)

  def __getitem__(self, idx):
    img_p = self.img_path[idx]
    img = PIL.Image.open(img_p)
    label = img_p.split('/')[-1].split('_')[-1].replace('.png','')
    label_idx =self.label_idx_dict[label]

    if self.transforms:
      img = self.transforms(img)

    return img, label_idx

  def __len__(self):
    return self.length

train_ds = CIFAR10Dataset(train_imgs, label_idx_dict, transforms=transforms_dict['test'])
test_ds = CIFAR10Dataset(test_imgs, label_idx_dict, transforms= transforms_dict['test'])

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=batch_size,shuffle=True, num_workers=num_workers)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers)

criterion = torch.nn.CrossEntropyLoss()


import torch.nn as nn

class tofp16(nn.Module):
    def __init__(self):
        super(tofp16, self).__init__()

    def forward(self, input):
        return input.half()

def copy_in_params(net, params):
    net_params = list(net.parameters())
    for i in range(len(params)):
        net_params[i].data.copy_(params[i].data)


def set_grad(params, params_with_grad):

    for param, param_w_grad in zip(params, params_with_grad):
        if param.grad is None:
            param.grad = torch.nn.Parameter(param.data.new().resize_(*param.data.size()))
        param.grad.data.copy_(param_w_grad.grad.data)

def BN_convert_float(module):
    if isinstance(module, torch.nn.modules.batchnorm._BatchNorm):
        module.float()
    for child in module.children():
        BN_convert_float(child)
    return module


def network_to_half(network):
    return nn.Sequential(tofp16(), BN_convert_float(network.half()))

In [0]:
lr=2e-3
num_epochs = 40
# name = 'MixNet-M'
name = 'ResNet50'
# name = 'MobileNet-v2'
# name='MnasNet-1'
# name = 'EfficientNet-B0'
# name = 'MobileNet-v3'

if name=='ResNet50':
  net = torchvision.models.resnet50(False)
  net.fc = torch.nn.Linear(2048, 10)
elif name=='MixNet-M':
  net = geffnet.mixnet_m(True)
  net.classifier = torch.nn.Linear(1536,10)
elif name=='ResNet18':
  net = torchvision.models.resnet18(True)
  net.fc = torch.nn.Linear(512,10)
elif name=='MobileNet-v2':
  net=torchvision.models.mobilenet_v2(True)
  net.classifier[1] = torch.nn.Linear(1280,10)
elif name=='MnasNet-1':
  net = torchvision.models.mnasnet1_0(False)
  net.classifier[1] = torch.nn.Linear(1280,10)
elif name=='EfficientNet-B0':
  net=EfficientNet.from_pretrained('efficientnet-b0')
  net._fc = torch.nn.Linear(1280,10)
# elif name=='MobileNet-v3':
#   net = MobileNetV3_Large()
#   weights = torch.load('mobilenetv3/mbv3_large.old.pth.tar')
#   weights = OrderedDict([(key.replace('module.',''),value) for key,value in weights['state_dict'].items()])
#   net.load_state_dict(weights)
#   net.linear4 = torch.nn.Linear(1280,10)

net = network_to_half(net)

optimizer = torch.optim.Adam(net.parameters(),lr=lr,eps=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

device = 'cuda:0' if torch.cuda.is_available else 'cpu'
torch.backends.cudnn.benchmark=False
net.to(device)
device

'cuda:0'

In [0]:
t = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
cur_time = str(t.month).zfill(2)+str(t.day).zfill(2) + str(t.hour).zfill(2)+str(t.minute).zfill(2)+str(t.second).zfill(2)

train_log_dir = os.path.join('logs/tensorboard/train',name,cur_time)
test_log_dir = os.path.join('logs/tensorboard/test',name,cur_time)

In [0]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
from google.colab import files
files.download("logs.zip")

In [0]:
print('START:',name)
for epoch in range(1, num_epochs+1):
  cur_lr = scheduler.get_lr()[0]
  epoch_losses = 0.
  epoch_corrects = 0.
  epoch_imgs = 0.

  since=time.time()

  net.train()
  for imgs,labels in train_dl:
    imgs,labels = imgs.to(device),labels.to(device)
    epoch_imgs += imgs.size(0)

    optimizer.zero_grad()
    outputs = net(imgs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    epoch_losses += loss.item() * imgs.size(0)
    epoch_corrects += sum(outputs.argmax(1) == labels).item()

  l = epoch_losses / epoch_imgs
  a = epoch_corrects / epoch_imgs
  print('Epoch {}: Loss {:.4f} Acc: {:.4f} lr:{:.4f} Time:{:.2f}'.format(epoch, l, a, cur_lr, time.time()-since))

  with SummaryWriter(train_log_dir) as writer:
    writer.add_scalar('loss',l,epoch)
    writer.add_scalar('acc',a,epoch)


  val_losses = 0.
  val_corrects = 0.
  val_imgs = 0.

  net.eval()
  for (imgs, labels) in test_dl:
    imgs,labels = imgs.to(device), labels.to(device)
    val_imgs += imgs.size(0)
    
    with torch.no_grad():
      outputs = net(imgs)
      loss = criterion(outputs, labels)

    val_losses += loss.item() * imgs.size(0)
    val_corrects+= sum(outputs.argmax(1) == labels).item()

  l = val_losses / val_imgs
  a = val_corrects / val_imgs

  print('Val: Loss: {:.4f} Acc: {:.4f}'.format(l, a))
  with SummaryWriter(test_log_dir) as writer:
    writer.add_scalar('loss', l, epoch)
    writer.add_scalar('acc',a,epoch)

  scheduler.step()

START: ResNet50
Epoch 1: Loss 2.3506 Acc: 0.2495 lr:0.0020 Time:25.61
Val: Loss: 3.1850 Acc: 0.3342
Epoch 2: Loss 1.9481 Acc: 0.3644 lr:0.0020 Time:25.46
Val: Loss: 4.4643 Acc: 0.3768
Epoch 3: Loss 1.7414 Acc: 0.4256 lr:0.0020 Time:25.41
Val: Loss: 1.4849 Acc: 0.4517
Epoch 4: Loss 1.5125 Acc: 0.4883 lr:0.0020 Time:25.52
Val: Loss: 1.5513 Acc: 0.4967
Epoch 5: Loss 1.3919 Acc: 0.5270 lr:0.0020 Time:25.56
Val: Loss: 1.7173 Acc: 0.5176
Epoch 6: Loss 1.3912 Acc: 0.5499 lr:0.0019 Time:25.68
Val: Loss: 1.3727 Acc: 0.5064
Epoch 7: Loss 1.3906 Acc: 0.5305 lr:0.0019 Time:25.80
Val: Loss: 1.4162 Acc: 0.5259
Epoch 8: Loss 1.2528 Acc: 0.5883 lr:0.0019 Time:25.75
Val: Loss: 1.2014 Acc: 0.5764
Epoch 9: Loss 1.1227 Acc: 0.6246 lr:0.0018 Time:25.55
Val: Loss: 1.8836 Acc: 0.5990
Epoch 10: Loss 0.9879 Acc: 0.6803 lr:0.0018 Time:25.53
Val: Loss: 1.1270 Acc: 0.6054
Epoch 11: Loss 0.8550 Acc: 0.7129 lr:0.0017 Time:25.91
Val: Loss: 1.5964 Acc: 0.6166
Epoch 12: Loss 0.8386 Acc: 0.7386 lr:0.0016 Time:25.63
Val

In [0]:
%tensorboard --logdir logs/tensorboard